# Shoe Training via Same CNN Pipeline as Clothing

In [1]:
# %load_ext autoreload
# %reload_ext autoreload

import numpy as np
from glob import glob
import pandas as pd

# import necessary keras modules
from keras.preprocessing import image 
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping  
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization

# custom functions
from layer_output import get_dense_layers, path_to_tensor, paths_to_tensor
from loading import load_files       

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [ ]:
# # define function to load train, test, and validation datasets
# def load_dataset(path):
#     data = load_files(path, ignore_files='.DS_Store')
#     shoe_files = np.array(data['filenames'])
#     shoe_targets = np_utils.to_categorical(np.array(data['target']), 9)
#     return shoe_files, shoe_targets

# # load train, test, and validation datasets
# train_files, train_targets = load_dataset('../data/shoes/train')
# valid_files, valid_targets = load_dataset('../data/shoes/validate')
# test_files, test_targets = load_dataset('../data/shoes/test')

# # load list of clothing names
# shoe_names = [item[20:-1] for item in sorted(glob("../data/shoes/train/*/"))]

# # print info about the dataset
# print(f'There are {len(shoe_names)} total shoe categories.')
# print(f'There are {len(np.hstack([train_files, valid_files, test_files]))} total shoe images.\n')
# print(f'There are {len(train_files)} training shoe images.')
# print(f'There are {len(valid_files)} validation shoe images.')
# print(f'There are {len(test_files)} test shoe images.')

In [2]:
tags = pd.read_pickle('saved_models/shoe_labels_df.pickle')
tags.head()

,type,path,shoe_type,color,class_11,class_12,class_13,class_14,class_15,class_16,...,class_75,class_76,class_77,class_81,class_82,class_83,class_84,class_85,class_86,class_87
0,validate,../data/shoes/validate/calf_boots/7838888.6.jpg,calf_boots,grape,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,validate,../data/shoes/validate/calf_boots/7677053.325.jpg,calf_boots,charcoal,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,validate,../data/shoes/validate/calf_boots/8024575.6357...,calf_boots,charcoal,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,validate,../data/shoes/validate/calf_boots/8075982.278.jpg,calf_boots,brownish,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,validate,../data/shoes/validate/calf_boots/8036333.84.jpg,calf_boots,charcoal,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
train_files = np.array(tags[tags['type']=='train']['path'])
valid_files = np.array(tags[tags['type']=='validate']['path'])
test_files = np.array(tags[tags['type']=='test']['path'])

train_targets = np.array(tags[tags['type']=='train'].loc[:,'class_11':]).astype('float32')
valid_targets =np.array( tags[tags['type']=='validate'].loc[:,'class_11':]).astype('float32')
test_targets = np.array(tags[tags['type']=='test'].loc[:,'class_11':]).astype('float32')
valid_targets

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
valid_targets.shape

In [4]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
# rescale the images by dividing every pixel in every image by 255.
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|██████████| 2013/2013 [00:02<00:00, 728.31it/s]


## Build the CNN Architechture

In [9]:
model = Sequential()

# Define the architecture.
model.add(BatchNormalization(input_shape=(224, 224, 3)))
model.add(Conv2D(filters=16, kernel_size=5, strides=2, padding='valid', activation='relu', 
                 input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32, kernel_size=5, strides=2, padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(rate=0.3))
model.add(Conv2D(filters=64, kernel_size=2, strides=2, padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(GlobalAveragePooling2D())

model.add(Dense(units=300, activation='relu', name='vectors')) # extract vectors from here and cluster
model.add(Dropout(rate=0.3))
model.add(Dense(56, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_2 (Batch (None, 224, 224, 3)       12        
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 110, 110, 16)      1216      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 55, 55, 16)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 26, 26, 32)        12832     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 6, 6, 64)          8256      
__________

In [10]:
# compile the model
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [7]:
# create and configure augmented image generator
datagen_train = ImageDataGenerator(
    width_shift_range=0.1,  # randomly shift images horizontally (10% of total width)
    height_shift_range=0.1,  # randomly shift images vertically (10% of total height)
    horizontal_flip=True) # randomly flip images horizontally

datagen_train.fit(train_tensors)

In [11]:
epochs = 20

early_stop = EarlyStopping(monitor='val_loss', patience=3)

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch_shoes2.hdf5', 
                               verbose=1, save_best_only=True)
history = model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=32, callbacks=[checkpointer, early_stop], verbose=1)


Train on 18038 samples, validate on 2164 samples
Epoch 1/20
18038/18038 [==============================] - 1673s 93ms/step - loss: 2.4906 - acc: 0.2669 - val_loss: 1.7728 - val_acc: 0.4552

Epoch 00001: val_loss improved from inf to 1.77283, saving model to saved_models/weights.best.from_scratch_shoes2.hdf5
Epoch 2/20
18038/18038 [==============================] - 1625s 90ms/step - loss: 1.7734 - acc: 0.4316 - val_loss: 1.6768 - val_acc: 0.4635

Epoch 00002: val_loss improved from 1.77283 to 1.67677, saving model to saved_models/weights.best.from_scratch_shoes2.hdf5
Epoch 3/20
18038/18038 [==============================] - 1631s 90ms/step - loss: 1.5468 - acc: 0.4956 - val_loss: 1.3620 - val_acc: 0.5684

Epoch 00003: val_loss improved from 1.67677 to 1.36200, saving model to saved_models/weights.best.from_scratch_shoes2.hdf5
Epoch 4/20
18038/18038 [==============================] - 1707s 95ms/step - loss: 1.4204 - acc: 0.5334 - val_loss: 1.2335 - val_acc: 0.5957

Epoch 00004: val_loss 

KeyboardInterrupt: 

## Graph of training/validation Accuracy and Loss

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.figure(figsize=(16,5))
plt.subplot(1,2,1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc', c='r')
plt.title('Training and validation accuracy')
plt.legend()
plt.subplot(1,2,2)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss', c='r')
plt.title('Training and validation loss')
plt.legend();

In [ ]:
# get index of predicted item for each image in test set
shoe_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(shoe_predictions)==np.argmax(test_targets, axis=1))/len(shoe_predictions)
print('Test accuracy: {}'.format(round(test_accuracy, 4)))

In [ ]:
# https://towardsdatascience.com/visualizing-intermediate-activation-in-convolutional
# -neural-networks-with-keras-260b36d60d0
model.save('saved_models/shoes_cnn_color.h5')
# load the model with best validation loss
model.load_weights('saved_models/weights.best.from_scratch_shoes_color.hdf5')


In [ ]:
img_path = '../data/shoes/test/sandals/7910998.332551.jpg'
img_tensor = path_to_tensor(img_path)
img_tensor /= 255.
plt.imshow(img_tensor[0])
plt.axis('off')
plt.show();

image = np.vstack([img_tensor])
classes = model.predict_classes(image)
print("Predicted class:", shoe_names[int(classes)])

## Get a matrix of all images Dense layer stacked

In [ ]:
train_files[:5]

In [13]:
# import pickle
# all_shoe_paths = np.append(train_files, np.append(valid_files, test_files))

# dense_layers, dense_df = get_dense_layers(model, all_shoe_paths)
# # pickle.dump(dense_df, open("saved_models/dense_shoe_df2.pickle", "wb" ))
# print(dense_layers.shape)
# print(dense_df.shape)
# dense_df.head()

### K-Means cluster graph

In [ ]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import pylab as pl

X = dense_layers
pca = PCA(n_components=2).fit(X)
pca_2d = pca.transform(X)
kmeans = KMeans(n_clusters=9, random_state=11)
kmeans.fit(X)
pl.figure()
pl.title('K-means with 9 clusters')
pl.scatter(pca_2d[:, 0], pca_2d[:, 1], c=kmeans.labels_, cmap='tab20');

### T-SNE Graph

In [ ]:
# https://datascience.stackexchange.com/questions/31700/how-to-print-kmeans-cluster-python
from sklearn.manifold import TSNE

# Embed the features into 2 features using TSNE
X_embedded_tsne = TSNE(n_components=2, perplexity=25).fit_transform(X)

plt.figure(figsize=(15,15))
plt.scatter(X_embedded_tsne[:,0], X_embedded_tsne[:,1], c = kmeans.labels_, cmap='tab20')
plt.title('TSNE of Shoe Targets')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.colorbar()
plt.show();
# plt.tight_layout();

**Assign each file to a cluster**

In [ ]:
# Assign each training file to a cluster for understanding the clusters and troubleshooting
for cluster_num in range(9):
    mask = np.where(kmeans.labels_ == cluster_num)[0]
    for idx in mask:
        print(f"Image {all_clothing_paths[idx]} is in cluster: {cluster_num}")

In [ ]:
# number of images in each cluster
[np.where(kmeans.labels_ == cluster_num, 1, 0).sum() for cluster_num in range(9)]

**Confusion matrix of labels and clusters**

In [ ]:
targets1 = np.array([np.where(target == 1)[0][0] for target in train_targets])
targets2 = np.array([np.where(target == 1)[0][0] for target in valid_targets])
targets3 = np.array([np.where(target == 1)[0][0] for target in test_targets])
targets = np.append(targets1, np.append(targets2, targets3))

In [ ]:
# create a confusion matrix out of the the labels and clusters
confusion = dict()

for target, label in zip(targets, kmeans.labels_):
    confusion[(target, label)] = confusion.get((target, label), 0) + 1

for target in range(9):
    line = '  '.join([f'{confusion.get((target,label),0):4d}' for label in range(9)])
    print(line)    

In [ ]:
for i, name in enumerate(shoe_names):
    print(i,': ', name)